Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""

---

In [ ]:
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import random
import torch.nn.functional as F
from sklearn.datasets import make_moons
from torch.utils.data import DataLoader, TensorDataset

! pip install pot

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_two_moons(n_samples=5000, noise=0.1):
    X, y = make_moons(n_samples=n_samples, noise=noise)
    X = (X - X.mean()) / X.std()
    return torch.tensor(X, dtype=torch.float32, device=device)

seed = 2025
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

x_data = load_two_moons(5000, noise=0.05)
dataset = TensorDataset(x_data)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
train_data_np = x_data.cpu().numpy()

x_data = load_two_moons(1000, noise=0.05)
val1_data_np = x_data.cpu().numpy()

x_data = load_two_moons(1000, noise=0.2)
val2_data_np = x_data.cpu().numpy()

In [ ]:
import ot

def calc_w_distance(X, Y):
    assert X.shape == Y.shape, "The shape of X and Y do not match"
    # Compute the pairwise squared Euclidean distance matrix (cost matrix)
    #M = np.sum((X[:, np.newaxis, :] - Y[np.newaxis, :, :]) ** 2, axis=2)
    M = ot.dist(X, Y, metric='euclidean')

    # Uniform weights (each point has equal probability)
    a = np.ones(X.shape[0]) / X.shape[0]
    b = np.ones(Y.shape[0]) / Y.shape[0]

    # Solve the optimal transport problem to compute W2 distance
    W2_distance = ot.emd2(a, b, M)  # Returns squared Wasserstein-2 distance
    #W2_distance = np.sqrt(W2_squared)  # Take square root to get W2

    print(f"Wasserstein-1 Distance: {W2_distance:.4f}")
    return W2_distance


def plot_two_moon(X, model_name):
    plt.figure(figsize=(4, 4))
    plt.scatter(X[:, 0], X[:, 1], alpha=0.5, edgecolors='k')
    plt.title(model_name)
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.tight_layout()
    plt.show()

In [ ]:
#plot_two_moon(train_data_np, "Training Data")
#plot_two_moon(val1_data_np, "Validation Data 1")
#plot_two_moon(val2_data_np, "Validation Data 2")

# NICE

In [ ]:
# DO NOT CHANGE THIS CELL
class StandardLogistic(torch.distributions.Distribution):
    """Standard logistic distribution.
    """
    def __init__(self):
        super(StandardLogistic, self).__init__(validate_args=False)

    def log_prob(self, x):
        """Computes data log-likelihood.
        Args:
            x: input tensor.
        Returns:
            log-likelihood.
        """
        return -(F.softplus(x) + F.softplus(-x))

    def sample(self, size):
        """Samples from the distribution.
        Args:
            size: number of samples to generate.
        Returns:
            samples.
        """
        z = torch.distributions.Uniform(0., 1.).sample(size)
        return torch.log(z) - torch.log(1. - z)

class AdditiveCouplingLayer(nn.Module):    
    def __init__(self, input_dim, hidden_dim, num_layers, partition):
        super().__init__()
        assert partition in ['odd', 'even']
        self.partition = partition
        _get_even = lambda xs: xs[:, 0::2]
        _get_odd = lambda xs: xs[:, 1::2]
        if (partition == 'even'):
            self._first = _get_even
            self._second = _get_odd
        else:
            self._first = _get_odd
            self._second = _get_even
        
        _modules = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        for _ in range(num_layers):
            _modules.append(nn.Linear(hidden_dim, hidden_dim) )
            _modules.append(nn.ReLU())
        _modules.append(nn.Linear(hidden_dim, input_dim) )
        self.net = nn.Sequential(*_modules)

    
    def forward(self, x):
        """Map an input through the partition and nonlinearity.
        y1 = x1
        y2 = x2 + m(x1)
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        return out

    def inverse(self, y):
        """Inverse mapping through the layer. Gradients should be turned off for this pass.
        x1 = y1
        x2 = y2 - m(y1)
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        return out


In [ ]:

class NICEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_m_layers, prior):
        super(NICEModel, self).__init__()
        assert (input_dim % 2 == 0)
        self.input_dim = input_dim
        # define network
        half_dim = int(input_dim / 2)
        self.layer1 = AdditiveCouplingLayer(half_dim, hidden_dim, num_m_layers, 'odd')
        self.layer2 = AdditiveCouplingLayer(half_dim, hidden_dim, num_m_layers, 'even')
        self.layer3 = AdditiveCouplingLayer(half_dim, hidden_dim, num_m_layers, 'odd')
        self.layer4 = AdditiveCouplingLayer(half_dim, hidden_dim, num_m_layers, 'even')
        self.scaling_diag = nn.Parameter(torch.zeros(1, input_dim), requires_grad=True)
        self.prior = prior
        self.register_buffer('_dummy', torch.empty([0, ]))

    def forward(self, x):
        """Forward pass through all invertible coupling layers.
        Args:
            x: Input data. float tensor of shape (batch_size, input_dim).
        Returns:
            z: Latent variable. float tensor of shape (batch_size, input_dim).
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        return z


    def inverse(self, z):
        """Invert a set of draws from logistic prior
        Args:
            z: Latent variable. float tensor of shape (batch_size, input_dim).
        Returns:
            x: Generated data. float tensor of shape (batch_size, input_dim).
        """
        with torch.no_grad():
            # YOUR CODE HERE
            raise NotImplementedError()
        return x
    
    def log_prob(self, x):
        """Computes data log-likelihood. (See Section 3.3 in the NICE paper.)
        Args:
            x: input minibatch.
        Returns:
            log_p: log-likelihood of input.
        """
        # YOUR CODE HERE
        raise NotImplementedError()
        return log_p
    
    def sample(self, num_samples):
        """Generates samples.
        Args:
            num_samples: number of samples to generate.
        Returns:
            x: samples from the data space X.
        """
        z = self.prior.sample((num_samples, self.input_dim)).to(self._dummy.device)
        # YOUR CODE HERE
        raise NotImplementedError()
        return x

In [ ]:
# DO NOT CHANGE THIS CELL
INPUT_DIM = 2
HIDDEN_DIM = 32
NUM_M_LAYERS = 3
MAX_ITERS = 20000

def inf_iterator(iterable):
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()
            
train_loader = inf_iterator(dataloader)

prior = StandardLogistic()
nice_ckpt = 'model_nice_2025.pt'
best_loss = np.inf
if not os.path.exists(nice_ckpt):
    model_nice = NICEModel(INPUT_DIM, HIDDEN_DIM, num_m_layers=NUM_M_LAYERS, prior=prior).to(device)
    train_op = optim.Adam(model_nice.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-4)

    model_nice.train()
    for i in range(1, MAX_ITERS + 1):
        batch = next(train_loader)
        batch = batch[0].to(device)

        log_p = model_nice.log_prob(batch)
        loss = -log_p.mean()

        train_op.zero_grad()
        loss.backward()
        train_op.step()
        if loss < best_loss:
            best_loss = loss
            print(f'iter: {i} loss: {loss:.4f}')
            best_model = copy.deepcopy(model_nice)
        if i % 1000 == 0:
            print(f'iter: {i} loss: {loss:.4f}')
    torch.save(best_model.state_dict(), nice_ckpt)

def sample_nice(model_nice, num_samples):
    samples = model_nice.sample(num_samples)
    return samples

In [ ]:
# DO NOT CHANGE THIS CELL
samples = sample_nice(best_model, 1000)
calc_w_distance(samples, val1_data_np)
calc_w_distance(samples, val2_data_np)
#plot_two_moon(samples, "NICE")

In [ ]:
# HIDDEN TEST CELL, DO NOT CHANGE

# WGAN

In [ ]:
# DO NOT CHANGE THIS CELL
import torch
import torch.nn as nn
# Feel free to use these variables
DIM = 512  # Model dimensionality
# Gaussian noise, as in the plots in the paper
LAMBDA = .1  # Smaller lambda seems to help for toy tasks specifically
CRITIC_ITERS = 5  # How many critic iterations per generator iteration
BATCH_SIZE = 128  # Batch size
ITERS = 500  # how many generator iterations to train for
one = torch.tensor(1, dtype=torch.float).to(device)
mone = one * -1
one = one.to(device)
mone = mone.to(device)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        main = nn.Sequential(
            nn.Linear(2, DIM),
            nn.ReLU(True),
            nn.Linear(DIM, DIM),
            nn.ReLU(True),
            nn.Linear(DIM, DIM),
            nn.ReLU(True),
            nn.Linear(DIM, 2),
        )
        self.main = main

    def forward(self, noise, real_data):
        output = self.main(noise)
        return output


class Discriminator(nn.Module):

    def __init__(self):
        super(Discriminator, self).__init__()

        main = nn.Sequential(
            nn.Linear(2, DIM),
            nn.ReLU(True),
            nn.Linear(DIM, DIM),
            nn.ReLU(True),
            nn.Linear(DIM, DIM),
            nn.ReLU(True),
            nn.Linear(DIM, 1),
        )
        self.main = main

    def forward(self, inputs):
        output = self.main(inputs)
        return output.view(-1)

In [ ]:
def calc_gradient_penalty(netD, real_data, fake_data):
    # YOUR CODE HERE
    raise NotImplementedError()
    return gradient_penalty

In [ ]:
wgan_ckpt = 'model_wgan_2025.pt'
seed = 2025
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
netG = Generator()
netD = Discriminator()

netD = netD.to(device)
netG = netG.to(device)

optimizerD = optim.Adam(netD.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr=1e-4, betas=(0.5, 0.9))

if not os.path.exists(wgan_ckpt):
    for iteration in range(ITERS):
        ############################
        # (1) Update D network
        ###########################
        # YOUR CODE HERE
        raise NotImplementedError()
            
        ############################
        # (2) Update G network
        ###########################
        # YOUR CODE HERE
        raise NotImplementedError()
    torch.save(netG.state_dict(), wgan_ckpt)

In [ ]:
def sample_wgan(netG, num_samples):
    """ Sampler from the WGAN model from an isotropic Gaussian distribution """
    # YOUR CODE HERE
    raise NotImplementedError()
    return samples

samples = sample_wgan(netG, 1000)
#plot_two_moon(samples, "WGAN")
calc_w_distance(samples, val1_data_np)
calc_w_distance(samples, val2_data_np)

In [ ]:
# HIDDEN TEST CELL, DO NOT CHANGE

# DDPM

In [ ]:
ddpm_ckpt = 'model_ddpm_2025.pt'
betas = torch.linspace(0.0001, 0.02, 100, device=device)

def q_sample(x_0, t, betas):
    """Adds noise to x_0 at timestep t."""

    # YOUR CODE HERE
    raise NotImplementedError()

class FeedForward(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=64):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(1, 10),
            nn.SiLU(),
            nn.Linear(10, input_dim)
        )
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim * 4),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x, t):
        x = self.encoder(x)
        x = self.decoder(x)
        return x + self.mlp(t.float().unsqueeze(-1))

def train_ddpm(model, betas, n_epochs=100, lr=2e-3):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    
    # YOUR CODE HERE
    raise NotImplementedError()
    torch.save(model.state_dict(), ddpm_ckpt)

model = FeedForward().to(device)
if not os.path.exists(ddpm_ckpt):
    train_ddpm(model, betas, n_epochs=100, lr=2e-3)

In [ ]:
# Generate new samples
alpha_cumprod = torch.sqrt((1 - betas).cumprod(dim=0))

def sample_ddpm(model, num_samples, betas):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
#samples = sample_ddpm(model, 1000, betas)
#plot_two_moon(samples, "DDPM")
#calc_w_distance(samples, val1_data_np)
#calc_w_distance(samples, val2_data_np)

In [ ]:
# HIDDEN TEST CELL, DO NOT CHANGE